In [1]:
import shutil
from glob import glob
import nrrd
import os
import PIL
from PIL import Image
from torch.utils.data import Dataset
import torch
import pandas as pd
import numpy as np
import matplotlib as plot
from matplotlib import pylab as plt
from preprocess.utils import *

In [2]:
import SimpleITK as sitk

"""
    以下x,y,z方向最大半径分割
    结合临床数据保存为numpy数组，[0]为切片数据,[1]为临床数据
"""
#读取excel文件
OUTSIZE=[32,32,32]
data_path=r'F:\Download\1'
csv_path=os.path.join(data_path,'results_20220822.xlsx')
data=pd.read_excel(csv_path,usecols='B:I')
exfiles=['09652616']
print(data)

          fileName  结节性质（纯=0，混合=1）  结节大小（≤1cm=0，＞1cm=1） 性别  年龄 有无吸烟  \
0    00625858.nrrd             0.0                  1.0  女  61    无   
1    01034481.nrrd             1.0                  0.0  女  47    无   
2    02501737.nrrd             1.0                  1.0  男  41    有   
3    03022694.nrrd             0.0                  0.0  男  47    有   
4    03223027.nrrd             0.0                  1.0  女  63    无   
..             ...             ...                  ... ..  ..  ...   
203  09823965.nrrd             0.0                  1.0  男  59    有   
204  09835955.nrrd             1.0                  1.0  女  49    无   
205  09836250.nrrd             1.0                  1.0  女  41    无   
206  68103460.nrrd             0.0                  0.0  男  44   有    
207  70063493.nrrd             1.0                  1.0  男  44    有   

          病理类型 EGFR突变情况  
0      右上肺原位腺癌        无  
1    右上肺微浸润性腺癌        无  
2     右上肺浸润性腺癌        有  
3     左肺微浸润性腺癌        有  
4    左上肺微浸润性腺癌   

In [6]:
#存在背景信息

error_file=[]
#image_path存放nrrd文件
image_path=os.path.join(data_path,'image')
for i in range(len(data)):
    filename=str(data.iloc[i,0])[:-5]
    cdata=np.array(data.iloc[i,:])
    if filename in exfiles:
        continue
    filepath=os.path.join(data_path,filename+'.nrrd')
    mask_file_path=os.path.join(data_path,filename+'_mask.nrrd')
    type=int(data.iloc[i,1])

    image_file=os.path.join(image_path,filename)
    origin_data=sitk.ReadImage(filepath)
    mask_data=sitk.ReadImage(mask_file_path)
    #若mask文件和nrrd文件不匹配，跳过
    if mask_data.GetSize()!=origin_data.GetSize():
        error_file.append(filename)
        continue
    #获取结节占据的x,y,z轴切片序号
    #注意sitk.GetArrayFromImage将图片又[x,y,z]变为[z,x,y]
    nonIndex=getnonzeros(sitk.GetArrayFromImage(mask_data))
    #设置窗口并缩放到[0,255]
    #origin_data=setWindow(origin_data)
    #连通域分割
    mask_data=sitk.ConnectedComponent(mask_data)
    label_shape_filter = sitk.LabelShapeStatisticsImageFilter()
    label_shape_filter.Execute(mask_data)
    #分割最大连通域
    area_max_label=0
    area_max=0
    for i in range(1, label_shape_filter.GetNumberOfLabels()+1):
        area=label_shape_filter.GetNumberOfPixels(i)
        if area>area_max:
            area_max_label = i
            area_max = area
    centor=origin_data.TransformPhysicalPointToIndex(label_shape_filter.GetCentroid(area_max_label))
    x,y,z,w,h,d=label_shape_filter.GetBoundingBox(area_max_label)
    r=int(max(w,h,d)*1.2)
    start_x,start_y,start_z=int(x-(r-w)/2),int(y-(r-h)/2),int(z-(r-d)/2)
    ROI_data=sitk.RegionOfInterest(origin_data,index=[start_x,start_y,start_z],size=[r,r,r])

    ROI_data=resample2Size(ROI_data,OUTSIZE)
    ndata=sitk.GetArrayFromImage(ROI_data)
    target_path=os.path.join(image_path,f'{type}')
    if not os.path.exists(target_path):
        os.makedirs(target_path)
    np.savez(os.path.join(target_path,filename+'.npz'),image=ndata,cdata=np.zeros(shape=(5,)))
    pic_path=os.path.join(image_path,'pic')
    if not os.path.exists(pic_path):
        os.makedirs(pic_path)
    ndata=zscore(ndata)*255
    img=Image.fromarray(ndata[int(OUTSIZE[2]/2),:,:])
    img.convert('L').save(os.path.join(pic_path,f'{filename}_z.png'))
    print(filename,':complete!')
print('错误文件：',error_file)

00625858 :complete!
01034481 :complete!
02501737 :complete!
03022694 :complete!
03223027 :complete!
03571004 :complete!
04183261 :complete!
04247827 :complete!
04595852 :complete!
04693158 :complete!
04776614 :complete!
04873120 :complete!
05118583 :complete!
05256788 :complete!
05376764 :complete!
05412252 :complete!
05431443 :complete!
05545138 :complete!
05587338 :complete!
05688162 :complete!
05738378 :complete!
05807742 :complete!
05849467 :complete!
05871891 :complete!
05930651 :complete!
05968948 :complete!
06025976 :complete!
06135499 :complete!
06137339 :complete!
06260590 :complete!
07056211 :complete!
07136826 :complete!
07156597 :complete!
07156637 :complete!
07157615 :complete!
07238217 :complete!
07280060 :complete!
07289679 :complete!
07459320 :complete!
07592297 :complete!
07723928 :complete!
07870634 :complete!
07942262 :complete!
08031582 :complete!
08127850 :complete!
08291767 :complete!
08350481 :complete!
08416480 :complete!
08516985 :complete!
08555483 :complete!


In [3]:
#不存在背景信息

error_file=[]
#image_path存放nrrd文件
image_path=os.path.join(data_path,'image1')
for i in range(len(data)):
    filename=str(data.iloc[i,0])[:-5]
    cdata=np.array(data.iloc[i,:])
    if filename in exfiles:
        continue
    filepath=os.path.join(data_path,filename+'.nrrd')
    mask_file_path=os.path.join(data_path,filename+'_mask.nrrd')
    type=int(data.iloc[i,1])

    image_file=os.path.join(image_path,filename)
    origin_data=sitk.ReadImage(filepath)
    mask_data=sitk.ReadImage(mask_file_path)
    #若mask文件和nrrd文件不匹配，跳过
    if mask_data.GetSize()!=origin_data.GetSize():
        error_file.append(filename)
        continue
    #获取结节占据的x,y,z轴切片序号
    #注意sitk.GetArrayFromImage将图片又[x,y,z]变为[z,x,y]
    nonIndex=getnonzeros(sitk.GetArrayFromImage(mask_data))
    #设置窗口并缩放到[0,255]
    origin_data=setWindow(origin_data)
    #连通域分割
    mask_data=sitk.ConnectedComponent(mask_data)
    label_shape_filter = sitk.LabelShapeStatisticsImageFilter()
    label_shape_filter.Execute(mask_data)
    #分割最大连通域
    area_max_label=0
    area_max=0
    for i in range(1, label_shape_filter.GetNumberOfLabels()+1):
        area=label_shape_filter.GetNumberOfPixels(i)
        if area>area_max:
            area_max_label = i
            area_max = area
    centor=origin_data.TransformPhysicalPointToIndex(label_shape_filter.GetCentroid(area_max_label))
    x,y,z,w,h,d=label_shape_filter.GetBoundingBox(area_max_label)
    r=max(w,h,d)
    #清楚背景信息
    Ndata=sitk.GetArrayFromImage(origin_data)
    Ndata_mask=sitk.GetArrayFromImage(mask_data)
    node_Ndata=Ndata*Ndata_mask
    node_Ndata[node_Ndata==0]=-1000
    node_data=sitk.GetImageFromArray(node_Ndata)
    start_x,start_y,start_z=int(x-(r-w)/2),int(y-(r-h)/2),int(z-(r-d)/2)
    ROI_data=sitk.RegionOfInterest(node_data,index=[start_x,start_y,start_z],size=[r,r,r])

    ROI_data=resample2Size(ROI_data,OUTSIZE)
    ndata=sitk.GetArrayFromImage(ROI_data)
    target_path=os.path.join(image_path,f'{type}')
    if not os.path.exists(target_path):
        os.makedirs(target_path)
    np.savez(os.path.join(target_path,filename+'.npz'),image=ndata,cdata=np.zeros(shape=(5,)))
    pic_path=os.path.join(image_path,'pic')
    if not os.path.exists(pic_path):
        os.makedirs(pic_path)
    ndata=zscore(ndata)*255
    img=Image.fromarray(ndata[int(OUTSIZE[2]/2),:,:])
    img.convert('L').save(os.path.join(pic_path,f'{filename}_z.png'))
    print(filename,':complete!')
print('错误文件：',error_file)


00625858 :complete!
01034481 :complete!
02501737 :complete!
03022694 :complete!
03223027 :complete!
03571004 :complete!
04183261 :complete!
04247827 :complete!
04595852 :complete!
04693158 :complete!
04776614 :complete!
04873120 :complete!
05118583 :complete!
05256788 :complete!
05376764 :complete!
05412252 :complete!
05431443 :complete!
05545138 :complete!
05587338 :complete!
05688162 :complete!
05738378 :complete!
05807742 :complete!
05849467 :complete!
05871891 :complete!
05930651 :complete!
05968948 :complete!
06025976 :complete!
06135499 :complete!
06137339 :complete!
06260590 :complete!
07056211 :complete!
07136826 :complete!
07156597 :complete!
07156637 :complete!
07157615 :complete!
07238217 :complete!
07280060 :complete!
07289679 :complete!
07459320 :complete!
07592297 :complete!
07723928 :complete!
07870634 :complete!
07942262 :complete!
08031582 :complete!
08127850 :complete!
08291767 :complete!
08350481 :complete!
08416480 :complete!
08516985 :complete!
08555483 :complete!
